In [1]:
!pip install --upgrade pip
!pip install faiss-cpu sentence-transformers transformers accelerate
!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install gradio
!pip install openai==0.28

  Using cached numpy-2.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Successfully uninstalled numpy-2.2.5
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.11.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.


In [2]:
import json
import faiss
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import openai

# ✅ OpenAI API 설정
openai.api_key = ""
MODEL_ID = "ft:gpt-3.5-turbo-0125:tset::BX2RnWfq"

# ✅ 저장된 데이터 로드
index = faiss.read_index("data/game_index.faiss")

with open("data/texts.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

with open("data/game_names.json", "r", encoding="utf-8") as f:
    game_names = json.load(f)

# ✅ 임베딩 모델 로딩 (RAG 쿼리용)
embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda")

# ✅ 출력 후처리 함수
def clean_output(raw_output: str) -> str:
    if "추천 완료!" in raw_output:
        raw_output = raw_output.split("추천 완료!")[0]
    return raw_output.strip()

# ✅ RAG 기반 추천 함수
def recommend_with_rag(query, default_k=3):
    number_match = re.search(r'(\d+)\s*개', query)
    top_k = int(number_match.group(1)) if number_match else default_k

    # 1. 쿼리 임베딩 → 유사도 검색
    query_embedding = embed_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(query_embedding), k=top_k)

    # 2. 검색된 게임 설명과 이름들 추출
    retrieved = []
    selected_names = []
    for i in I[0]:
        name = game_names[i]
        desc = texts[i]
        retrieved.append(f"[{name}]\n{desc}")
        selected_names.append(f"- {name}")

    context = "\n\n".join(retrieved)
    name_list_str = "\n".join(selected_names)

    # 3. 프롬프트 생성
    prompt = f"""아래는 보드게임 설명입니다. 각 게임은 "[게임명]\n설명" 형식으로 되어 있습니다.

[게임 설명]
{context}

⚠️ 반드시 아래의 게임 이름 목록 중에서만 선택할 수 있습니다. 이 목록에 없는 게임 이름을 절대 생성하지 마세요.
만약 목록에 없는 게임명을 출력하면 실패로 간주됩니다.

[허용된 게임 이름 목록]
{ name_list_str }

[사용자 질문]
{query}

📌 출력 지침:
- 반드시 위 목록에 있는 게임 중에서만 3개를 골라 추천하세요.
- 출력 형식은 반드시 아래 형식처럼 작성하세요:

게임명1: 추천 이유
게임명2: 추천 이유
게임명3: 추천 이유

- 각 줄은 '게임명: 추천 이유' 형식으로만 작성하고, 줄바꿈 이외에 아무 포맷도 쓰지 마세요.
- 추천이 모두 끝나면 마지막 줄에 반드시 다음과 같이 써주세요:
추천 완료!

그 이후에는 아무 것도 쓰지 마세요.
"""

    # ✅ GPT-3.5 Turbo 파인튜닝 모델 호출
    response = openai.ChatCompletion.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "너는 보드게임 추천 도우미야. 사용자의 질문에 따라 관련 게임을 추천하고 이유도 알려줘."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=768
    )

    raw_output = response["choices"][0]["message"]["content"]
    cleaned = clean_output(raw_output)

    print("\n🤖 추천게임 (RAG 기반):\n")
    print(cleaned)

    return cleaned


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
import json
import faiss
import numpy as np
import os
import openai
from sentence_transformers import SentenceTransformer

# ✅ OpenAI API 설정
openai.api_key = ""
MODEL_ID = "ft:gpt-3.5-turbo-0125:tset::BX2RnWfq"

# ✅ 출력 후처리 함수
def clean_output(raw_output: str) -> str:
    return raw_output.strip()

# ✅ game.json 로딩
with open("data/game.json", "r", encoding="utf-8") as f:
    game_data = json.load(f)
game_names = [g["game_name"] for g in game_data]

# ✅ 게임 이름 입력
selected_game = input("🎯 어떤 게임의 룰을 보고 싶나요? 정확한 게임명을 입력해주세요:\n").strip()
if selected_game not in game_names:
    print(f"❌ '{selected_game}' 게임을 찾을 수 없습니다.")
    exit()

# ✅ 전체 룰 불러오기
game_info = next(g for g in game_data if g["game_name"] == selected_game)
game_rule_text = game_info['text']

# ✅ 벡터 경로 설정
base_path = "data/game_data/game_data"
faiss_path = os.path.join(base_path, f"{selected_game}.faiss")
chunks_path = os.path.join(base_path, f"{selected_game}.json")

# ✅ 존재 확인
for path in [faiss_path, chunks_path]:
    if not os.path.exists(path):
        print(f"❌ '{selected_game}'에 대한 벡터 파일이 없습니다: {path}")
        exit()

# ✅ 룰 요약 먼저 출력
print("\n🧠 AI가 룰을 분석하여 설명 중...\n")
response = openai.ChatCompletion.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": (
            "너는 보드게임 룰 전문 AI야. 반드시 아래 규칙을 따라야 해:\n"
            "- 사용자가 선택한 보드게임의 룰 전체를 보고, 그 게임의 룰을 알기 쉽게 설명해줘.\n"
            "- 핵심 개념, 목표, 진행 방식, 승리 조건을 요약해줘.\n"
            "- 설명은 간결하고 구조적으로 작성해."
        )},
        {"role": "user", "content": f"게임 이름: {selected_game}\n\n룰 전체:\n{game_rule_text}\n\n이 게임의 룰을 설명해주세요."}
    ],
    temperature=0.7,
    max_tokens=768
)

initial_explanation = response["choices"][0]["message"]["content"]
print(f"\n📘 게임 '{selected_game}'의 룰 설명:\n{clean_output(initial_explanation)}")

# ✅ 벡터 인덱스 및 청크 텍스트 로딩
index = faiss.read_index(faiss_path)
with open(chunks_path, "r", encoding="utf-8") as f:
    chunks = json.load(f)

# ✅ 임베딩 모델 로딩
embed_model = SentenceTransformer("BAAI/bge-m3")

# ✅ 세부 질문 반복 함수
def answer_loop(selected_game, user_q):
    if user_q.lower() == "q":
        return "❌ 'q'는 종료 명령입니다. Gradio에서는 사용할 수 없습니다."
    
    q_vec = embed_model.encode([user_q], normalize_embeddings=True)
    faiss_path = os.path.join("data/game_data/game_data", f"{selected_game}.faiss")
    chunks_path = os.path.join("data/game_data/game_data", f"{selected_game}.json")

    if not os.path.exists(faiss_path) or not os.path.exists(chunks_path):
        return f"❌ '{selected_game}'에 대한 벡터 또는 청크 파일이 존재하지 않습니다."

    index = faiss.read_index(faiss_path)
    with open(chunks_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)

    D, I = index.search(np.array(q_vec), k=3)
    retrieved_chunks = [chunks[i] for i in I[0]]
    
    context = "\n\n".join(retrieved_chunks)
    rag_prompt = f"""
아래는 '{selected_game}' 보드게임의 룰 설명 일부입니다:

{context}

이 룰을 바탕으로 다음 질문에 정확하고 구체적으로 답변해줘:

질문: {user_q}
"""

    response = openai.ChatCompletion.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": (
                "너는 보드게임 룰 전문 AI야. 반드시 아래 규칙을 따라야 해:\n"
                "- 사용자의 질문에 대해 아래 룰 설명(context)에 있는 내용만 기반해서 답변해.\n"
                "- 룰 설명에 없는 정보는 절대로 지어내거나 상상하지 마.\n"
                "- 사람 이름, 장소, 시간, 인원수 등을 추측하거나 새로 만들어내지 마.\n"
                "- 답할 수 없는 질문이면 '해당 정보는 룰에 명시되어 있지 않습니다.' 라고 말해."
            )},
            {"role": "user", "content": rag_prompt}
        ],
        temperature=0.7,
        max_tokens=768
    )

    answer = response["choices"][0]["message"]["content"]
    return clean_output(answer)

# ✅ 콘솔용 질문 루프
while True:
    user_q = input("\n❓ 룰에 대해 더 궁금한 점이 있다면 질문해주세요 (종료: q): ").strip()
    if user_q.lower() == "q":
        break
    answer = answer_loop(selected_game, user_q)
    print(f"\n💬 답변:\n{answer}")


🎯 어떤 게임의 룰을 보고 싶나요? 정확한 게임명을 입력해주세요:
 뱅



🧠 AI가 룰을 분석하여 설명 중...


📘 게임 '뱅'의 룰 설명:
게임 이름: 뱅

룰 설명:
1. **게임 목적**: 보안관, 부관, 무법자, 배신자 중 누군가가 목표를 이루면 게임에서 승리합니다. 총알 토큰(생명)을 모두 잃은 사람은 게임에서 탈락합니다.
   
2. **게임 세팅**: 각 플레이어는 직업 카드와 인물 카드를 받고, 총알 토큰을 가져옵니다. 직업에 따라 특정 목표를 달성해야 합니다. 게임 시작 시 보안관부터 시작하여 시계방향으로 플레이합니다.

3. **게임 진행**: 턴이 돌아오면 3가지 행동을 할 수 있습니다. (1) 사용 카드 2장을 뽑습니다. (2) 사용 카드를 내려 써서 특수 능력을 발동시킵니다. (3) 총알 토큰을 초과하는 사용 카드는 버려져야 합니다.

4. **공격과 방어**: 특정 카드를 사용하여 다른 플레이어를 공격하거나, 공격을 피할 수 있습니다. 거리를 조절하여 공격 범위를 결정할 수 있습니다.

5. **카드 종류**: 다양한 사용 카드가 있으며, 각 카드에는 특정 효과가 있습니다. 카드를 잘 활용하여 전략을 세워야 합니다.

6. **게임 종료**: 특정 직업의 플레이어가 목표를 달성하거나, 특정 플레이어가 탈락할 때까지 게임을 진행합니다.

이렇게 뱅 게임은 각 플레이어의 직업과 능력을 활용하여 상대를 제거하거나 목표를 달성하는 게임으로, 전략적인 카드 사용과 상황 판단이 중요한 게임입니다.



❓ 룰에 대해 더 궁금한 점이 있다면 질문해주세요 (종료: q):  7명이서 플레이할때 초기세팅을 알려줘



💬 답변:
7명이서 '뱅' 보드게임을 플레이할 때의 초기세팅은 다음과 같습니다:
1. 보안관 카드 1장, 무법자 카드 3장, 배신자 카드 1장, 부관 카드 2장을 모아 종류별로 섞습니다.
2. 섞인 직업 카드를 플레이어 수(7명)에 맞게 1장씩 분배합니다.
3. 각 플레이어는 자신이 받은 직업 카드를 확인한 후, 보안관 카드만 공개합니다.
4. 인물 카드를 모아 섞고, 1장씩 각 플레이어에게 분배합니다.
5. 받은 인물 카드는 모두 공개되며, 각 플레이어는 자신이 가진 인물의 능력을 모두에게 알려줍니다.
6. 인물 카드에 있는 총알 수만큼 총알 토큰을 해당 플레이어가 가져갑니다. 보안관은 1개를 더 가져갑니다.
7. 각 플레이어는 총알 토큰의 개수만큼 사용 카드를 뒷면이 보이게 놓고, 나머지 사용 카드는 중앙에 놓습니다.
8. 게임 진행을 위한 매뉴얼도 각 플레이어에게 1개씩 분배합니다.

이러한 과정을 거치면 7명이서 '뱅' 보드게임을 시작할 준비가 됩니다.



❓ 룰에 대해 더 궁금한 점이 있다면 질문해주세요 (종료: q):  첫 턴 진행을 도와줘



💬 답변:
첫 턴의 경우 보안관이 선공을 가집니다. 보안관은 다음과 같이 행동합니다:
1. 카드 더미에서 사용 카드 2장을 가져옵니다.
2. 자신이 가진 사용 카드를 보고 원하는 만큼 카드를 사용합니다. 단, 한 차례에 뱅 카드는 1장만 사용할 수 있습니다.
3. 자신이 가진 총알 토큰(생명)을 초과하는 사용 카드를 가지고 있다면 버려야 합니다.

이후 보안관의 차례가 종료되면 시계방향으로 다음 플레이어의 차례가 시작됩니다.



❓ 룰에 대해 더 궁금한 점이 있다면 질문해주세요 (종료: q):  q


In [4]:
import gradio as gr

# ✅ Gradio UI 정의
with gr.Blocks() as demo:
    with gr.Tab("게임 추천"):
        gr.Markdown("🕹️ 여기에 게임 추천 기능을 추가하세요.")
        
        with gr.Row():
            query_input = gr.Textbox(
                label="❓ 원하는 게임을 설명해주세요",
                placeholder="예: 두 명이 할 수 있는 전략적인 게임 3개 추천해줘"
            )
    
    recommend_button = gr.Button("추천받기 🎮")
    output_box = gr.Textbox(label="🤖 추천 결과", max_lines=50)

    recommend_button.click(fn=recommend_with_rag, inputs=query_input, outputs=output_box)

    with gr.Tab("룰 질의응답"):
        gr.Markdown("🎲 게임을 선택하면 요약된 룰을 보여주고, 추가 질문을 할 수 있어요.")
    
        selected_game = gr.Dropdown(choices=game_names, label="게임 이름 선택")
    
        rule_summary_box = gr.Textbox(label="📘 선택한 게임의 룰 요약", lines=10, interactive=False)
    
        question_input = gr.Textbox(label="❓ 룰에 대한 질문", placeholder="예: 몇 명이 할 수 있나요?")
        ask_btn = gr.Button("질문하기")
        answer_output = gr.Textbox(label="💬 AI의 답변", lines=5, interactive=False)
    
        # ✅ 게임 선택 시 룰 요약 함수
        def summarize_rule(selected_game):
            game_info = next(g for g in game_data if g["game_name"] == selected_game)
            game_rule_text = game_info['text']
    
            response = openai.ChatCompletion.create(
                model=MODEL_ID,
                messages=[
                    {"role": "system", "content": (
                        "너는 보드게임 룰 전문 AI야. 반드시 아래 규칙을 따라야 해:\n"
                        "- 사용자가 선택한 보드게임의 룰 전체를 보고, 그 게임의 룰을 알기 쉽게 설명해줘.\n"
                        "- 핵심 개념, 목표, 진행 방식, 승리 조건을 요약해줘.\n"
                        "- 설명은 간결하고 구조적으로 작성해."
                    )},
                    {"role": "user", "content": f"게임 이름: {selected_game}\n\n룰 전체:\n{game_rule_text}\n\n이 게임의 룰을 설명해주세요."}
                ],
                temperature=0.7,
                max_tokens=768
            )
            return clean_output(response["choices"][0]["message"]["content"])
    
        # ✅ 자동 요약 연결: 게임 선택하면 바로 요약 출력
        selected_game.change(fn=summarize_rule, inputs=selected_game, outputs=rule_summary_box)
    
        # ✅ 질문하기 연결
        ask_btn.click(fn=answer_loop, inputs=[selected_game, question_input], outputs=answer_output)


demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://16470762b573acb787.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
